## Pull in UDFs

In [ ]:
%run nb_udfs

## Run Reports

In [ ]:
workspace = 'Admin%20Center' #have to escape the & symbol and spaces
lakehouse = 'lh_monitoring'

report_table = 'dimReports'

In [ ]:
#get a list of all reports and load to a table in the lakehouse
response = _base_api(
        request=f"/v1.0/myorg/admin/reports",
        method="get"
    )
df_reports = pd.json_normalize(response.json()['value'])
df_reports = spark.createDataFrame(df_reports)
df_reports = df_reports.drop("users","subscriptions","endorsementDetails.endorsement")
#df_reports.printSchema()
#print(df_reports.columns)


In [ ]:
#creates a slowly changing dimension so we can keep an eye on any deleted reports
reportPath = udf_GetFilePath(workspace, lakehouse, report_table)
#print(reportPath)
naturalKeyColumnList = ['id']
primaryKeyColumnName = "tableId"
returnVal = udf_UpsertDimension(df_reports,2,reportPath,naturalKeyColumnList,primaryKeyColumnName,True)
print(returnVal)